# Genetic Algorithm

### Neural Net - PyTorch

In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import random

from torch.autograd import Variable
torch.manual_seed(420)

In [2]:
import pandas as pd

data = pd.read_csv('/Users/veda/Downloads/Bank_Personal_Loan_Modelling.csv')

In [3]:
data.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [4]:
data.drop(["ID", "ZIP Code"],axis=1,inplace=True)

In [5]:
x = data['Personal Loan']
data = data.drop(['Personal Loan'],axis=1)
data = data.join(x)

In [6]:
data.shape

(5000, 12)

In [7]:
data.isnull().sum()

Age                   0
Experience            0
Income                0
Family                0
CCAvg                 0
Education             0
Mortgage              0
Securities Account    0
CD Account            0
Online                0
CreditCard            0
Personal Loan         0
dtype: int64

In [8]:
x = data.iloc[:,:-1].values
y = data.iloc[:,-1].values
x.shape,y.shape

((5000, 11), (5000,))

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state=69)
(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

((4000, 11), (1000, 11), (4000,), (1000,))

In [10]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [11]:
X_train = torch.from_numpy(x_train).to(torch.float32)
Y_train = torch.from_numpy(y_train).to(torch.float32)
print(X_train)
print(Y_train)

tensor([[ 0.4146,  0.3461,  0.4544,  ..., -0.2515,  0.8216, -0.6453],
        [ 0.7620,  0.8670,  0.4112,  ..., -0.2515,  0.8216, -0.6453],
        [ 1.5436,  1.4746,  1.4262,  ..., -0.2515, -1.2171,  1.5496],
        ...,
        [-1.2355, -1.2164,  2.2899,  ..., -0.2515, -1.2171, -0.6453],
        [ 1.5436,  1.6482, -0.3446,  ..., -0.2515,  0.8216, -0.6453],
        [-0.0197, -0.0879, -0.7333,  ...,  3.9758,  0.8216,  1.5496]])
tensor([0., 0., 0.,  ..., 0., 0., 0.])


In [12]:
BATCH_SIZE = 32
LEARNING_RATE = 0.003
EPOCH = 100

In [13]:
from torch.utils.data import DataLoader, TensorDataset

data = TensorDataset(X_train,Y_train)
data = DataLoader(data,batch_size=BATCH_SIZE,shuffle=True)

In [14]:
for i in data:
    print(i)

[tensor([[-6.2761e-01, -6.0873e-01,  1.9228e+00, -1.2119e+00,  1.2332e+00,
         -1.0411e+00, -5.6215e-01, -3.4024e-01, -2.5152e-01, -1.2171e+00,
         -6.4531e-01],
        [ 1.3699e+00,  1.3010e+00, -1.1004e+00,  5.2592e-01, -9.3755e-01,
          1.3311e+00, -5.6215e-01, -3.4024e-01, -2.5152e-01,  8.2161e-01,
         -6.4531e-01],
        [-8.8815e-01, -8.6915e-01,  1.3182e+00,  5.2592e-01,  1.6330e+00,
          1.4500e-01, -5.6215e-01, -3.4024e-01, -2.5152e-01,  8.2161e-01,
         -6.4531e-01],
        [-1.4961e+00, -1.5636e+00,  1.6205e+00, -3.4301e-01,  3.0040e+00,
         -1.0411e+00, -5.6215e-01, -3.4024e-01, -2.5152e-01,  8.2161e-01,
         -6.4531e-01],
        [-9.7500e-01, -9.5596e-01,  6.4876e-01,  5.2592e-01, -4.2343e-01,
          1.3311e+00, -5.6215e-01, -3.4024e-01, -2.5152e-01,  8.2161e-01,
         -6.4531e-01],
        [-1.4092e+00, -1.3032e+00, -8.6285e-01,  1.3949e+00, -8.8042e-01,
          1.4500e-01, -5.6215e-01, -3.4024e-01, -2.5152e-01,  8.2161e-

In [15]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(X_train.shape[1], X_train.shape[1])
        self.relu = nn.ReLU()
        self.l2 = nn.Linear( X_train.shape[1],1)
        self.sigmoid = nn.Sigmoid()
    def forward(self,val):
        val = self.l1(val)
        val = self.relu(val)
        val = self.l2(val)
        val = self.sigmoid(val)
        return val    

In [16]:
model = NeuralNet()
model

NeuralNet(
  (l1): Linear(in_features=11, out_features=11, bias=True)
  (relu): ReLU()
  (l2): Linear(in_features=11, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [17]:
loss_function = nn.BCELoss()
# for BCELoss to be used the output should be passed to Sigmoid function
optimizer = torch.optim.Adam(model.parameters(),lr=LEARNING_RATE)

In [18]:
training_loss = [0]*EPOCH
training_accuracy = [0]*EPOCH

for i in range(1,EPOCH+1):
    epoch_loss = 0
    accuracy = 0
    for x_batch,y_batch in data:
        # x_batch,y_batch = x_batch.to(device),y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(x_batch)
        
        loss = loss_function(y_pred,y_batch.unsqueeze(1))
        loss.backward()
        
        optimizer.step()
        epoch_loss += loss.item() * len(x_batch)
        cnt = (torch.where(y_pred>=0.5, 1, 0) == y_batch.unsqueeze(1)).sum().float()
        accuracy += cnt.item()
    if(i%10 == 0):
        print(i,epoch_loss/len(data.dataset),(accuracy)/len(data.dataset))

10 0.06676904394850135 0.976
20 0.05513021308928728 0.9805
30 0.04945240390487015 0.9825
40 0.04552915525995195 0.98575
50 0.042996242824941876 0.98525
60 0.041324632616713645 0.9855
70 0.03997949011810124 0.9865
80 0.03867880521295592 0.986
90 0.03826634056679905 0.98775
100 0.03711410496151075 0.9865


In [19]:
X_test = torch.from_numpy(x_test).to(torch.float32)
Y_test = torch.from_numpy(y_test).to(torch.float32)

In [20]:
test = TensorDataset(X_test,Y_test)
test = DataLoader(test,batch_size=1)

In [21]:
y_pred_t = model(X_test)
y_pred_t = torch.where(y_pred_t>=0.5, 1, 0).flatten()
accuracy = (y_pred_t == Y_test).sum().float().item() / len(test.dataset)
print(accuracy)

0.977


### GA Weight Optimisation

In [22]:
def fitness(model):
    y_pred = model(X_train)
    y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
    accuracy = (y_pred == Y_train).sum().float().item() / len(test.dataset)
    return accuracy

In [29]:
def crossover(m1,m2):
    # Shape of the weights and biases
    shw = [i.numpy().shape for i in m1.parameters()]
    shb = [i[0]*i[1] if len(i) == 2 else i[0] for i in shp]
    
    
    m_p_1 = np.concatenate([i.numpy().flatten() for i in m1.parameters()])
    m_p_2 = np.concatenate([i.numpy().flatten() for i in m2.parameters()])
    
    start_index = len(m_p_1)//2 - 0
    end_index = len(m_p_1)//2 + 10
    mid_value_index = random.randrange(start_index,end_index)
    
    c_1 = np.concatenate([m_p_1[:mid_value_index], m_p_2[mid_value_index:]])
    c_2 = np.concatenate([m_p_2[:mid_value_index], m_p_1[mid_value_index:]])
    
    return (m_p_1,c_1,c_2,shb)


In [ ]:
def mutation(m_p_1,c_1,c_2,shb):
    
    
    r_start = random.randrange(0,len(m_p_1)//2)
    r_end = random.randrange(r_start,len(m_p_1))
    
    c_1_mutate = c_1.copy()
    c_1_mutate[r_start:r_end] = c_1_mutate[r_start:r_end][::-1]
    
    # Child 2 Mutation
    random_start = random.randrange(0,len(matrix_1)//2)
    random_end = random.randrange(random_start,len(matrix_1))
    
    c_2_mutate = c_2.copy()
    c_2_mutate[r_start:r_end] = c_2_mutate[r_start:r_end][::-1]
    
    # Converting the array to parameters
    c = [c_1, c_2, c_1_mutate, c_2_mutate]
    output = list()
    
    for i in c:
        param = list()
        cum_sum = 0
        for j in range(len(shb)):
            array = i[cum_sum : cum_sum + shb[i]]
            array = array.reshape(shp[i])
            cum_sum += shb[i]
            param.append(array)
        param = np.array(param, dtype="object")
        output.append(param)
    
    output = np.array(output, dtype="object")
    return output

In [30]:
population_size = 10

def train(no):
    # Initial Population
    population = np.array([NeuralNet() for i in range(population_size)])
    
    best_model = None
    
    for loop in range(no):
        # Sort population based up on fitness function
        population = population[np.argsort([fitness(model) for model in population])]
        
        # Printing Max Accuracy
        best_model = population[-1]
        print("Gen", loop+1, " :", fitness(population[-1]))
        
        # Parent Selection
        parent_max_1 = population[-1]
        parent_max_2 = population[-2]
        parent_max_3 = population[-3]
        parent_max_4 = population[-4] 
        parent_1 = population[0]
        parent_2 = population[1]
        
        # Crossover
        a,b,x,y = crossover(parent_max_1, parent_max_3)
        output1 = mutation(a,b,x,y)
        c,d,w,z = crossover(parent_max_2, parent_max_4)
        output2 = mutation(c,d,w,z)
        output = np.concatenate([output_1, output_2])
        
        # New population
        new_population = np.array([NeuralNet() for i in range(len(output))])
        for count, model in enumerate(new_population, 0):
            for index, param in enumerate(model.parameters(), 0):
                param.data = (torch.tensor(output[count][index]))
        
        # Add Poor performing parent to the new population
        new_population = np.concatenate([new_population, [parent_1, parent_2]])
        
        # New population becomes your next population
        population = new_population.copy()
        
    return best_model

In [28]:
best_model = train(10)

Gen 1  : 3.564


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.